In [ ]:
import numpy as np
import os
import concurrent.futures
from tqdm import tqdm
import pandas as pd
from privacy_estimates import AttackResults

# sys path hack
import sys; sys.path.insert(0, '..')
from utils.exp import prep_exp
from utils.audit import compute_eps_lower_single
from run_attack import run_exp

# NOTE: this is the number of models for D only. so total number of models is 2 * (2000 + 1000 + 2000) = 10000.
# NOTE: Also when doing cross validation, the split is re-calculated for each repetition, so you can ignore this as long as the total is correct
n_shadow, n_valid, n_test = 2000, 1000, 2000
data_names = ['adult', 'fire']
model_names = ['DPartPB', 'DSynthPB', 'MST', 'NIST_MST', 'DPWGAN', 'DPWGANCity']
n_procs = 32

def results_mean_std(resultss):
    # calculate mean and standard deviation of results
    all_results = pd.concat(resultss)

    cols_of_interest = ['emp_eps', 'auc', 'emp_eps_gdp', 'emp_eps_approxdp']
    rest_cols = list(set(all_results.columns).difference(cols_of_interest))

    all_results = all_results.groupby(rest_cols).agg({col: [np.mean, np.std] for col in cols_of_interest}).reset_index()
    all_results.columns = [f'{i}_{j}' if j != '' else i for i, j in all_results.columns]

    return all_results

## Querybased vs DCR (Black-box)

In [ ]:
resultss = []
results_folder = 'results/cv_shuffle_results/qb_vs_dcr/'
for rep in tqdm(range(5)):
    results_path = f'{results_folder}/rep_{rep}.csv'
    # load old results
    if os.path.exists(results_path):
        results_df = pd.read_csv(results_path)
        records = results_df.to_dict('records')
    else:
        results_df = None
        records = []

    with concurrent.futures.ProcessPoolExecutor(max_workers=n_procs) as executor:
        futures = {}
        for data_name in data_names:
            for model_name in model_names:
                for epsilon in [1.0, 4.0]:
                    curr_model_name = f'{model_name}_eps{epsilon}'
                    neighbour = 'edit' if 'PB' in model_name else 'addremove'
                    out_dir = f'exp_data/qb_vs_dcr/{data_name}/{neighbour}'
                    if not os.path.exists(f'{out_dir}/df_out/{curr_model_name}/model_4999/synth_df.csv.gz'):
                        continue

                    df_in, df_out, metadata = prep_exp(out_dir=out_dir)

                    # analyze if not previously analyzed
                    for attack_type in ['bb_querybased', 'bb_dcr']:
                        if results_df is None or len(results_df[(results_df['data_name'] == data_name) &
                                                                (results_df['model'] == model_name) &
                                                                (results_df['theor_eps'] == epsilon) &
                                                                (results_df['attack_type'] == attack_type)]) == 0:
                            futures[executor.submit(run_exp, curr_model_name, df_in, df_out, metadata, n_shadow, n_valid,
                                                    n_test, out_dir, attack_type=attack_type, cv_shuffle_id=rep)] = data_name

        with tqdm(total=len(futures)) as pbar:
            for future in concurrent.futures.as_completed(futures):
                record = future.result()

                # add metadata
                record['data_name'] = futures[future]

                records.append(record)
                pbar.update(1)

    results_df = pd.DataFrame.from_records(records)

    # sort results according to data_name, model, attack_type
    results_df = results_df.sort_values(['data_name', 'model', 'attack_type', 'theor_eps'], ignore_index=True)

    # re-order columns to ['data_name', 'model', 'attack_type', ...]
    cols = list(results_df.columns)
    cols.remove('data_name')
    cols.insert(0, 'data_name')
    results_df = results_df[cols]

    results_df.to_csv(results_path, index=False)

    resultss.append(results_df)

results_df = results_mean_std(resultss)
results_df.to_csv(f'{results_folder}/results_agg.csv', index=False)
results_df

In [ ]:
# look at rows where QB has violation but not DCR
# select settings where QB has violation
sel_rows = ((results_df['attack_type'] == 'bb_querybased') & (results_df['emp_eps_mean'] > results_df['theor_eps']))
results_df = results_df.merge(results_df[sel_rows], on=['theor_eps', 'model', 'data_name'], suffixes=('', '_right'))[['attack_type', 'theor_eps', 'model', 'data_name', 'emp_eps_mean', 'auc_mean']]

# select settings where DCR does not have violation
results_df = results_df.merge(results_df[results_df['emp_eps_mean'] < results_df['theor_eps']], on=['theor_eps', 'model', 'data_name'], suffixes=('', '_right'))[['attack_type', 'theor_eps', 'model', 'data_name', 'emp_eps_mean', 'auc_mean']]

# sort values so that it is easy to read
results_df.sort_values(['attack_type', 'theor_eps', 'model', 'data_name'])

In [ ]:
# PrivBayes (Hazy)
# get scores for Querybased and DCR
from run_attack import get_scoress
import dill

epsilon = 1.0

scoresss = {}
with concurrent.futures.ProcessPoolExecutor(max_workers=n_procs) as executor:
    futures = {}
    for data_name in data_names:
        for model_name in ['DPartPB']:
            curr_model_name = f'{model_name}_eps{epsilon}'
            neighbour = 'edit' if 'PB' in model_name else 'addremove'
            out_dir = f'exp_data/qb_vs_dcr/{data_name}/{neighbour}'
            if not os.path.exists(f'{out_dir}/df_out/{curr_model_name}/model_4999/synth_df.csv.gz'):
                continue

            df_in, df_out, metadata = prep_exp(out_dir=out_dir)

            futures[executor.submit(get_scoress, curr_model_name, df_in, df_out, metadata, n_shadow, n_valid, n_test, out_dir, attack_type='bb_querybased')] = (data_name, 'querybased')
            futures[executor.submit(get_scoress, curr_model_name, df_in, df_out, metadata, n_shadow, n_valid, n_test, out_dir, attack_type='bb_dcr')] = (data_name, 'dcr')

    with tqdm(total=len(futures)) as pbar:
        for future in concurrent.futures.as_completed(futures):
            curr_scoress = future.result()

            # add metadata
            data_name, attack_type = futures[future]

            if data_name not in scoresss:
                scoresss[data_name] = {}
            
            scoresss[data_name][attack_type] = curr_scoress

            pbar.update(1)

dill.dump(scoresss, open('results/qb_vs_dcr.dill', 'wb'))

## Worst-case Dataset
- comparing different types of worst-case
- comparing black-box vs white-box

In [ ]:
resultss = []
results_folder = 'results/cv_shuffle_results/test_worst_case/'
for rep in tqdm(range(5)):
    results_path = f'{results_folder}/rep_{rep}.csv'
    data_name = 'adult'
    epsilon = 4.0

    # load old results
    if os.path.exists(results_path):
        results_df = pd.read_csv(results_path)
        records = results_df.to_dict('records')
    else:
        results_df = None
        records = []

    with concurrent.futures.ProcessPoolExecutor(max_workers=n_procs) as executor:
        futures = {}
        for model_name in model_names:
            curr_model_name = f'{model_name}_eps{epsilon}'
            for suffix in ['worstcase', 'worstcase_narrow', 'worstcase_repeat', 'worstcase_narrow_repeat']:
                neighbour = 'edit' if 'PB' in model_name else 'addremove'

                out_dir = f'exp_data/test_worst_case/adult/{neighbour}'
                
                df_in, df_out, metadata = prep_exp(out_dir=out_dir)

                # skip if model data has not been generated yet
                if not os.path.exists(f'{out_dir}/df_out/{curr_model_name}/model_4999/synth_df.csv.gz'):
                    continue
                
                # analyze if not previously analyzed
                for attack_type in ['bb_querybased', 'wb']:
                    if results_df is None or len(results_df[(results_df['model'] == model_name) &
                                                            (results_df['theor_eps'] == epsilon) &
                                                            (results_df['attack_type'] == attack_type) &
                                                            (results_df['dataset_type'] == suffix)]) == 0:
                        futures[executor.submit(run_exp, curr_model_name, df_in, df_out, metadata, n_shadow, n_valid,
                                                n_test, out_dir, attack_type=attack_type, cv_shuffle_id=rep)] = suffix

        with tqdm(total=len(futures), leave=False) as pbar:
            for future in concurrent.futures.as_completed(futures):
                record = future.result()
                
                # add metadata to record
                record['dataset_type'] = futures[future]

                records.append(record)
                pbar.update(1)

    results_df = pd.DataFrame.from_records(records)

    # sort results according to model, attack_type, dataset_type
    results_df = results_df.sort_values(['model', 'attack_type', 'dataset_type'], ignore_index=True)

    # re-order columns to ['model', 'attack_type', 'dataset_type', ...]
    cols = list(results_df.columns)
    cols.remove('dataset_type')
    cols.insert(2, 'dataset_type')
    results_df = results_df[cols]

    results_df.to_csv(results_path, index=False)

    resultss.append(results_df)

results_df = results_mean_std(resultss)
results_df.to_csv(f'{results_folder}/results_agg.csv', index=False)
results_df

In [ ]:
# white-box vs black-box
results_df = pd.read_csv(f'results/cv_shuffle_results/test_worst_case/results_agg.csv')
results_df = results_df[
    ((results_df['model'] == 'DSynthPB') & (results_df['dataset_type'] == 'worstcase_narrow')) |
    ((results_df['model'] == 'DPartPB') & (results_df['dataset_type'] == 'worstcase_narrow_repeat')) | 
    ((results_df['model'].isin(['NIST_MST', 'MST', 'DPWGAN', 'DPWGANCity'])) & (results_df['dataset_type'] == 'worstcase_repeat'))
]
results_df.sort_values(['model'])

## Worst-case Dataset
- implementation-specific
- comparing different epsilons

In [ ]:
resultss = []
results_folder = 'results/cv_shuffle_results/worst_case_wb/'
for rep in tqdm(range(5)):
    results_path = f'{results_folder}/rep_{rep}.csv'
    data_name = 'adult'

    # load old results
    if os.path.exists(results_path):
        results_df = pd.read_csv(results_path)
        records = results_df.to_dict('records')
    else:
        results_df = None
        records = []

    with concurrent.futures.ProcessPoolExecutor(max_workers=2) as executor:
        futures = {}
        for model_name in ['DPartPB']:
            for epsilon in [1.0, 2.0, 4.0, 10.0]:
                curr_model_name = f'{model_name}_eps{epsilon}'

                neighbour = 'edit' if 'PB' in model_name else 'addremove'
                out_dir = f'exp_data/worst_case_wb/adult/{neighbour}_worstcase'
                if model_name == 'DSynthPB':
                    out_dir = f'{out_dir}_narrow'
                elif model_name == 'DPartPB':
                    out_dir = f'{out_dir}_narrow_repeat'
                else:
                    out_dir = f'{out_dir}_repeat'

                df_in, df_out, metadata = prep_exp(out_dir=out_dir)

                # skip if model data has not been generated yet
                if not os.path.exists(f'{out_dir}/df_out/{curr_model_name}/model_4999/'):
                    continue
                
                # analyze if not previously analyzed
                if results_df is None or len(results_df[(results_df['model'] == model_name) &
                                                        (results_df['theor_eps'] == epsilon)]) == 0:
                    futures[executor.submit(run_exp, curr_model_name, df_in, df_out, metadata, n_shadow, n_valid, n_test,
                                            out_dir, attack_type='wb', cv_shuffle_id=rep)] = None

        with tqdm(total=len(futures), leave=False) as pbar:
            for future in concurrent.futures.as_completed(futures):
                record = future.result()
                records.append(record)
                pbar.update(1)

    results_df = pd.DataFrame.from_records(records)

    # sort results according to model, attack_type, dataset_type
    results_df = results_df.sort_values(['model', 'theor_eps'], ignore_index=True)
    results_df.to_csv(results_path, index=False)

    resultss.append(results_df)

results_df = results_mean_std(resultss)
results_df.to_csv(f'{results_folder}/results_agg.csv', index=False)
results_df

## Active White-box Attack

In [ ]:
resultss = []
results_folder = 'results/cv_shuffle_results/active_wb/'
for rep in tqdm(range(5)):
    results_path = f'{results_folder}/rep_{rep}.csv'
    data_name = 'adult'

    # load old results
    if os.path.exists(results_path):
        results_df = pd.read_csv(results_path)
        records = results_df.to_dict('records')
    else:
        results_df = None
        records = []

    with concurrent.futures.ProcessPoolExecutor(max_workers=n_procs) as executor:
        futures = {}
        for model_name in ['DPWGAN']:
            for epsilon in [1.0, 2.0, 4.0, 10.0]:
                for suffix in ['worstcase_repeat']:
                    curr_model_name = f'{model_name}_eps{epsilon}'

                    neighbour = 'edit' if 'PB' in model_name else 'addremove'
                    out_dir = f'exp_data/active_wb/adult/{neighbour}_{suffix}'
                    df_in, df_out, metadata = prep_exp(out_dir=out_dir)

                    # skip if model data has not been generated yet
                    if not os.path.exists(f'{out_dir}/df_out/{curr_model_name}/model_4999/'):
                        continue
                    
                    # analyze if not previously analyzed
                    for attack_type in ['bb_querybased', 'wb', 'active_wb']:
                        if results_df is None or len(results_df[(results_df['model'] == model_name) &
                                                                (results_df['theor_eps'] == epsilon) &
                                                                (results_df['attack_type'] == attack_type)]) == 0:
                            futures[executor.submit(run_exp, curr_model_name, df_in, df_out, metadata, n_shadow, n_valid,
                                                    n_test, out_dir, attack_type=attack_type, shuffle_seed=rep)] = suffix

        with tqdm(total=len(futures), leave=False) as pbar:
            for future in concurrent.futures.as_completed(futures):
                record = future.result()

                # add metadata to record
                record['dataset_type'] = futures[future]

                records.append(record)
                pbar.update(1)

    results_df = pd.DataFrame.from_records(records)

    # sort results according to model, attack_type, dataset_type
    results_df = results_df.sort_values(['model', 'theor_eps', 'attack_type', 'dataset_type'], ignore_index=True)
    results_df.to_csv(results_path, index=False)

    resultss.append(results_df)

results_df = results_mean_std(resultss)
results_df.to_csv(f'{results_folder}/results_agg.csv', index=False)
results_df

## Bugs
- DataSynthesizer v0.1.4

In [ ]:
resultss = []
results_folder = 'results/cv_shuffle_results/ds_0.1.4'
for rep in tqdm(range(5)):
    results_path = f'{results_folder}/rep_{rep}.csv'
    data_name = 'adult'
    n_shadow, n_valid, n_test = 2000, 1000, 2000

    # load old results
    if os.path.exists(results_path):
        results_df = pd.read_csv(results_path)
        records = results_df.to_dict('records')
    else:
        results_df = None
        records = []

    with concurrent.futures.ProcessPoolExecutor(max_workers=n_procs) as executor:
        futures = {}
        model_name = 'DSynthPB'
        for epsilon in [1.0, 2.0, 4.0, 10.0]:
            curr_model_name = f'{model_name}_eps{epsilon}'

            neighbour = 'edit' if 'PB' in model_name else 'addremove'
            out_dir = f'exp_data/ds_0.1.4/adult/{neighbour}_worstcase_narrow'
            df_in, df_out, metadata = prep_exp(out_dir=out_dir)

            # skip if model data has not been generated yet
            if not os.path.exists(f'{out_dir}/df_out/{curr_model_name}/model_4999/'):
                continue
            
            # analyze if not previously analyzed
            if results_df is None or len(results_df[(results_df['model'] == model_name) &
                                                    (results_df['theor_eps'] == epsilon)]) == 0:
                futures[executor.submit(run_exp, curr_model_name, df_in, df_out, metadata, n_shadow, n_valid, n_test,
                                        out_dir, attack_type='wb', shuffle_seed=rep)] = None

        with tqdm(total=len(futures), leave=False) as pbar:
            for future in concurrent.futures.as_completed(futures):
                record = future.result()
                records.append(record)
                pbar.update(1)

    results_df = pd.DataFrame.from_records(records)

    # sort results according to model, attack_type, dataset_type
    results_df = results_df.sort_values(['model', 'theor_eps'], ignore_index=True)
    results_df.to_csv(results_path, index=False)

    resultss.append(results_df)

results_df = results_mean_std(resultss)
results_df.to_csv(f'{results_folder}/results_agg.csv', index=False)
results_df

## DPWGAN
- early stopping bug

In [ ]:
resultss = []
results_folder = 'results/cv_shuffle_results/dpwgan_bug'
for rep in tqdm(range(5)):
    results_path = f'{results_folder}/rep_{rep}.csv'
    data_name = 'adult'
    n_shadow, n_valid, n_test = 2000, 1000, 2000

    # load old results
    if os.path.exists(results_path):
        results_df = pd.read_csv(results_path)
        records = results_df.to_dict('records')
    else:
        results_df = None
        records = []

    with concurrent.futures.ProcessPoolExecutor(max_workers=n_procs) as executor:
        futures = {}
        model_name = 'DPWGAN'
        for epsilon in [0.1, 0.4, 1.0, 4.0]:
            curr_model_name = f'{model_name}_eps{epsilon}'

            neighbour = 'edit' if 'PB' in model_name else 'addremove'
            out_dir = f'exp_data/dpwgan_bug/adult/{neighbour}_worstcase'
            df_in, df_out, metadata = prep_exp(out_dir=out_dir)

            # skip if model data has not been generated yet
            if not os.path.exists(f'{out_dir}/df_out/{curr_model_name}/model_4999/'):
                continue
            
            # analyze if not previously analyzed
            for attack_type in ['bb_querybased', 'wb', 'active_wb']:
                if results_df is None or len(results_df[(results_df['model'] == model_name) &
                                                        (results_df['theor_eps'] == epsilon) &
                                                        (results_df['attack_type'] == attack_type)]) == 0:
                    futures[executor.submit(run_exp, curr_model_name, df_in, df_out, metadata, n_shadow, n_valid, n_test,
                                            out_dir, attack_type=attack_type, shuffle_seed=rep)] = None

        with tqdm(total=len(futures), leave=False) as pbar:
            for future in concurrent.futures.as_completed(futures):
                record = future.result()
                records.append(record)
                pbar.update(1)

    results_df = pd.DataFrame.from_records(records)

    # sort results according to model, attack_type, dataset_type
    results_df = results_df.sort_values(['model', 'attack_type', 'theor_eps'], ignore_index=True)
    results_df.to_csv(results_path, index=False)

    resultss.append(results_df)

results_df = results_mean_std(resultss)
results_df.to_csv(f'{results_folder}/results_agg.csv', index=False)
results_df

## Extras

In [ ]:
resultss = []
results_folder = 'results/cv_shuffle_results/feat_types_epses/'
for rep in tqdm(range(5)):
    results_path = f'{results_folder}/rep_{rep}.csv'
    data_name = 'adult'
    n_shadow, n_valid, n_test = 2000, 1000, 2000

    # load old results
    if os.path.exists(results_path):
        results_df = pd.read_csv(results_path)
        records = results_df.to_dict('records')
    else:
        results_df = None
        records = []

    with concurrent.futures.ProcessPoolExecutor(max_workers=n_procs) as executor:
        futures = {}
        for model_name in ['DPartPB', 'DSynthPB', 'NIST_MST', 'MST']:
            for epsilon in [1.0, 2.0, 4.0, 10.0]:
                curr_model_name = f'{model_name}_eps{epsilon}'

                neighbour = 'edit' if 'PB' in model_name else 'addremove'
                out_dir = f'exp_data/worst_case_wb/adult/{neighbour}_worstcase'
                if model_name == 'DSynthPB':
                    out_dir = f'{out_dir}_narrow'
                elif model_name == 'DPartPB':
                    out_dir = f'{out_dir}_narrow_repeat'
                else:
                    out_dir = f'{out_dir}_repeat'
                df_in, df_out, metadata = prep_exp(out_dir=out_dir)

                # skip if model data has not been generated yet
                if not os.path.exists(f'{out_dir}/df_out/{curr_model_name}/model_4999/'):
                    continue
                    
                for feat_type in ['vals', 'errors+sum']:
                    # analyze if not previously analyzed
                    if results_df is None or len(results_df[(results_df['model'] == model_name) &
                                                            (results_df['theor_eps'] == epsilon) &
                                                            (results_df['attack_type'] == f'wb_{feat_type}')]) == 0:
                        futures[executor.submit(run_exp, curr_model_name, df_in, df_out, metadata, n_shadow, n_valid,
                                                n_test, out_dir, attack_type=f'wb_{feat_type}', shuffle_seed=rep)] = None

        with tqdm(total=len(futures), leave=False) as pbar:
            for future in concurrent.futures.as_completed(futures):
                record = future.result()

                records.append(record)
                pbar.update(1)

    results_df = pd.DataFrame.from_records(records)

    # sort results according to model, attack_type, dataset_type
    results_df = results_df.sort_values(['model', 'theor_eps', 'attack_type'], ignore_index=True)
    results_df.to_csv(results_path, index=False)
    
    resultss.append(results_df)

results_df = results_mean_std(resultss)
results_df.to_csv(f'{results_folder}/results_agg.csv', index=False)
results_df

In [ ]:
# difference in maximum auditable eps
n_obs = 2000
cp_eps = compute_eps_lower_single(AttackResults(FP=0, FN=0, TP=n_obs//2, TN=n_obs//2), 0.1, 0, method='cp')
zb_eps = compute_eps_lower_single(AttackResults(FP=1, FN=1, TP=n_obs//2-1, TN=n_obs//2-1), 0.1, 0, method='zb')
cp_eps, zb_eps